In [4]:
import pandas as pd

from src.logger import logging

In [5]:
try:
    course_info_df = pd.read_csv('data/365_course_info.csv')
    course_ratings_df = pd.read_csv('data/365_course_ratings.csv')
    exam_info_df = pd.read_csv('data/365_exam_info.csv')
    quiz_info_df = pd.read_csv('data/365_quiz_info.csv')
    student_engagement_df = pd.read_csv('data/365_student_engagement.csv')
    student_exams_df = pd.read_csv('data/365_student_exams.csv')
    student_hub_questions_df = pd.read_csv('data/365_student_hub_questions.csv')
    student_info_df = pd.read_csv('data/365_student_info.csv')
    student_learning_df = pd.read_csv('data/365_student_learning.csv')
    student_purchases_df = pd.read_csv('data/365_student_purchases.csv')
    student_quizzes_df = pd.read_csv('data/365_student_quizzes.csv')

    logging.info('Loaded dataframes')

except Exception as e:
    logging.exception(e)